In [1]:
import pandas as pd

from main import ParseReport, fuzz

report = ParseReport(file="result/TEST - Magellan - Cloud Example and Explanation.json", json_file=True, save_json=False)
report.get_lien()

2023-08-09 15:53:44.759531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 15:53:45.316505: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-09 15:53:45.316548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-09 15:53:45.316554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0


[{'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$87,052.04',
  'Recorded Date:': '01/10/2020',
  'Recording Information:': None,
  'Comment:': None},
 {'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$35,986.01',
  'Recorded Date:': '03/08/2010',
  'Recording Information:': None,
  'Comment:': None}]

In [2]:
report.words.head()

,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,line_geometry,word_idx,value,confidence,x1,y1,x2,y2
index,,,,,,,,,,,,,,,,,,
0,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",0,Southern,0.999931,0.436523,0.056641,0.545898,0.074219
1,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",1,Indiana,0.962854,0.552734,0.056641,0.643555,0.074219
2,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",2,Abstract,0.979812,0.649414,0.058594,0.750977,0.073242
3,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",3,"Company,",0.980987,0.756836,0.057617,0.879883,0.078125
4,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",4,Inc,0.987708,0.886719,0.055664,0.926758,0.075195


In [3]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
        
line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
# line = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
# line = context[(context['page_idx'] == line.page_idx.item()) & (context['block_idx'] == line.block_idx.item()) & (context['line_idx'] == line.line_idx.item())]["value"].values
line

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,score
index,,,,,,,,,,,,
2374,13,222,517,0.424805,0.083984,0.485352,0.098633,Notice,Notice Lien,0.133789,0.000000,0
2021,12,187,427,0.420898,0.089844,0.479492,0.101562,Notice,Notice Lien,0.134766,0.001465,0
3020,15,269,681,0.250977,0.227539,0.312500,0.238281,Notices,"Notices Lien,",0.191406,0.092285,8


In [4]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]

line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
result = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result


array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [5]:
result = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result

array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [6]:
word1, word2 = "Unpaid", "Balance"
row = line.iloc[1][["page_idx", "block_idx", "line_idx", "paired"]]
c = report.words[(report.words.page_idx == row.page_idx)]
column = c[c['value'].str.contains(word1) | c['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
column['paired']=  column.value + " " + column.value.shift(-1)
column['space']=  abs(column.x1.shift(-1) - column.x2)
column['align']=  (abs(column.y1.shift(-1) - column.y1) + (column.y2.shift(-1) - column.y2))/2

column['x12']=  column.x1.shift(-1)
column['y12']=  column.y1.shift(-1)
column['x22']=  column.x2.shift(-1)
column['y22']=  column.y2.shift(-1)

column["score"] = column["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
column = column[column.score < 10]
column = column.sort_values(by=['score', 'space', "align"])
column

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,x12,y12,x22,y22,score
index,,,,,,,,,,,,,,,,
2222,12,201,467,0.731445,0.34082,0.779297,0.352539,Unpaid,Unpaid Balance,0.000977,-0.000488,0.780273,0.34082,0.830078,0.351562,0


In [7]:
c[(c.block_idx==231)][["value", "line_idx"]]

,value,line_idx
index,,


In [8]:
right = 0.03
left = 0.03
column_data = c[(c.x1 >= column.x1.item() - left) & (c.x2 <= column.x22.item() + right) & (c.y1 >= column.y2.item())]
column_data['hd']=  abs(column_data.y1.shift(-1) - column.y2.item())
column_data = column_data.sort_values(by=['hd'])
column_data['bid']=  abs(column_data.block_idx.shift(-1) - column_data.block_idx)
column_data['lid']=  abs(column_data.line_idx.shift(-1) - column_data.line_idx)
column_data['h2d']=  abs(column_data.hd.shift(-1) - column_data.hd)
column_data = column_data.sort_values(by=['hd', 'h2d', 'bid', "lid"])
column_data = column_data.reset_index(drop=True)
column_data['hd2']=  column_data.y1.shift(-1) - column_data.y1
values = []

for i, k in column_data.iterrows():
    print(k.value, k.bid, k.lid, k.h2d)
    print(k.bid - column_data.iloc[i + 1].bid.item() > 0)
    print(k.lid - column_data.iloc[i + 1].lid.item() > 0)
    print(column_data.iloc[i + 1].h2d.item() > 0.12)
    print(i)

    if k.hd > 0.2 or column_data.iloc[i + 1].h2d.item() > 0.2:
    # print(column_data.bid - column_data.iloc[i + 1].bid.item())
    # print(k.bid - column_data.iloc[i + 1].bid.item())
        break
    else:
        values.append(k.value)

# values
column_data[["value", "block_idx", "line_idx", "hd2", 'hd', 'h2d', 'bid', "lid"]]

(f) 0.0 1.0 0.0126953125
False
False
True
0
25423.95 0.0 1.0 0.189453125
False
False
False
1
10562.06 5.0 8.0 0.0009765625
True
True
False
2


/tmp/ipykernel_106238/1242024779.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_data['hd']=  abs(column_data.y1.shift(-1) - column.y2.item())


,value,block_idx,line_idx,hd2,hd,h2d,bid,lid
0,(f),201,469,0.011719,0.019531,0.012695,0.0,1.0
1,25423.95,201,470,0.012695,0.032227,0.189453,0.0,1.0
2,10562.06,201,471,0.189453,0.221680,0.000977,5.0,8.0
3,$,206,479,0.049805,0.222656,0.047852,1.0,2.0
4,on,207,481,-0.048828,0.270508,0.000977,0.0,1.0
5,35986.01,207,480,0.047852,0.271484,0.068359,0.0,1.0
6,"this,",207,481,0.095703,0.339844,0.024414,7.0,19.0
7,Tax,214,500,0.020508,0.364258,0.000977,1.0,1.0
8,60025X,213,499,-0.020508,0.365234,0.000000,1.0,1.0
9,of,214,500,-0.000977,0.365234,0.000977,0.0,0.0
